<a href="https://colab.research.google.com/github/notmanan/Depression-Detection-Through-Multi-Modal-Data/blob/master/Rf_prune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import nltk
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import classification_report

print("Text Results: ")

dataset1 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/MCA Dataset/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


countPos = 0
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

Data = []
Y = []

Data_test = []
Y_test = []
index = -1
for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    if(val == 0 and countPos>38):
        continue
    Y.append(val)
    index+=1
    if(Y[index] == 0):
        countPos+=1

    
    fileName = "/content/drive/My Drive/MCA Dataset/train_data/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])


for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    if(val == 0):
        continue
    Y.append(val)
    fileName = "/content/drive/My Drive/MCA Dataset/dev_data/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
    Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])
    
for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    fileName = "/content/drive/My Drive/MCA Dataset/test_data/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
    Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8'))[:, 2:4])

    
Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)
    
for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)
        
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/MCA Dataset/GoogleNews-vectors-negative300.bin', binary=True)

stop_words = set(stopwords.words('english'))


def remove_StopWOrds(sentence):
    filtered_sentence = [] 
    for w in sentence: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1
    
    return float(correct)/len(Y_pred)


max_num_words = 15
max_num_sentence = 200

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words))
print(finalMatrix.shape)
for k in range(Data2.shape[0]):
    for i in range(min(max_num_sentence, len(Data2[k][0]))):
    	sentence = Data2[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X = np.zeros((len(Data2), max_num_sentence*max_num_words))
for i in range(len(finalMatrix)):
    X[i] = finalMatrix[i].flatten().reshape(1, -1)
 
#Test_data
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words))
print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    for i in range(min(max_num_sentence, len(Data2_test[k][0]))):
    	sentence = Data2_test[k][i].split(" ")
    	sentence = remove_StopWOrds(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    			finalMatrix_test[k][i][j] = np.average(np.array(model[sentence[j]]))
    		except:
    			continue
X_test = np.zeros((len(Data2_test), max_num_sentence*max_num_words))
for i in range(len(finalMatrix_test)):
    X_test[i] = finalMatrix_test[i].flatten().reshape(1, -1)

# from sklearn.ensemble import RandomForestClassifier
# clf1 = RandomForestClassifier()
# clf1.fit(X,Y)
# Y_pred_rf_1 = clf1.predict(X_test)
# print("Random forests: ")
# print(clf1.classification_report(Y_test,Y_pred_rf_1))
# print("Train: "+ str(clf1.score(X,Y)))
# print("Test: " + str(clf1.score(X_test,Y_test)))

from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# print(random_grid)


from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=1, random_state=42, n_jobs = -1)




rf_random.fit(X, Y)


from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
max_depth = rf_random.best_params_['max_depth']
max_features = rf_random.best_params_['max_features']
min_samples_leaf = rf_random.best_params_['min_samples_leaf']
min_samples_split  = rf_random.best_params_['min_samples_split'] 
n_estimators = rf_random.best_params_['n_estimators'] 

param_grid = {
    'bootstrap': [rf_random.best_params_['bootstrap']],
    'max_depth': [max_depth - 10,max_depth, max_depth + 10],
    'max_features': [2, 3],
    'min_samples_leaf': [min_samples_leaf - 1, min_samples_leaf, min_samples_leaf + 1],
    'min_samples_split': [min_samples_split - 2, min_samples_split, min_samples_split + 2],
    'n_estimators': [100, abs(n_estimators - 50), n_estimators, n_estimators + 50, 2*n_estimators]
}
rf = RandomForestClassifier()


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 1)

grid_search.fit(X, Y)
print("FINALLY CORRECT") 
print(grid_search.best_params_)
print("Random forests: ")
Y_pred_rf_1 = grid_search.predict(X_test)
print(classification_report(Y_test,Y_pred_rf_1))
print("Train: "+ str(grid_search.score(X,Y)))
print("Test: " + str(grid_search.score(X_test,Y_test)))

